In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
def format_json(data):
    init = pd.DataFrame.from_dict(data).loc[0][0]
    for obj in init:
        if type(init[obj])!=list or len(init[obj])!=1:
            init[obj] = [init[obj]]
    df = pd.DataFrame.from_dict(init)
    for i in range(1,pd.DataFrame.from_dict(data).shape[0]):
        curr = pd.DataFrame.from_dict(data).loc[i][0]
        for obj in curr:
            if type(curr[obj])!=list or len(curr[obj])!=1:
                curr[obj] = [curr[obj]]
        df_curr = pd.DataFrame.from_dict(curr)
        df = df.append(df_curr)
    return df

In [3]:
with open("EvaluationMedical.json", "r", encoding='utf8') as read_file:
    data = json.load(read_file)

In [ ]:
med = format_json(data)
med

In [ ]:
med.columns

In [ ]:
med_drop = list(med.isnull().sum()[med.isnull().sum(axis=0)>0.8*med.shape[0]].index)
med_drop

In [ ]:
med.drop(med_drop,axis=1,inplace=True)

In [ ]:
med.columns

In [ ]:
client = []
for c in med['client']:
    client.append(c['objectId'])
med['objectId'] = client
med.drop('client',axis=1,inplace=True)
med.columns

In [ ]:
med.drop(['createdAt','updatedAt'],axis=1,inplace=True)

In [ ]:
med.columns

In [ ]:
lat = []
lon = []
for h in med.location.values:
    lat.append(h['latitude'])
    lon.append(h['longitude'])

In [ ]:
pd.Series(lat).value_counts()

In [ ]:
pd.Series(lon).value_counts()

In [ ]:
med.drop('location',axis=1,inplace=True)

In [ ]:
med.columns

In [ ]:
med['AssessmentandEvaluation'].value_counts()

In [ ]:
medeval = []
for m in med['AssessmentandEvaluation']:
    try:
        if 'No' in m:
            medeval.append("No")
        else: medeval.append("Yes")
    except:
        medeval.append("No")
med['AssessmentandEvaluation'] = medeval
med['AssessmentandEvaluation'].value_counts()

In [ ]:
med['planOfAction'].value_counts()[:30]

In [ ]:
poa = []
for p in med['planOfAction']:
    p = str(p).lower()
    if 'orto' in p:
        poa.append("orthopedics")
    elif ('general' in p  or 'genral' in p) and 'med' in p:
        poa.append("general medicine")
    elif 'cardio' in p:
        poa.append("cardiology")
    elif ('general' in p or 'genral' in p) and 'cirug' in p:
        poa.append("general surgery")
    elif 'urol' in p:
        poa.append("urology")
    elif 'neur' in p:
        poa.append("neurology")
    elif 'derma' in p:
        poa.append("dermatology")
    elif 'ginec' in p:
        poa.append("gynecology")
    elif 'ofta' in p or 'opta' in p:
        poa.append("ophthalmology")
    elif 'otor' in p:
        poa.append("laryngology")
    elif 'gastro' in p:
        poa.append("gastroenterology")
    elif 'pedia' in p:
        poa.append("pediatrics")
    elif ('no' in p and 'neces' in p) or 'none' in p:
        poa.append("none")
    else: poa.append("other")
pd.Series(poa).value_counts()
med['planOfAction'] = poa
med['planOfAction'].value_counts()

In [ ]:
med['notes'].value_counts()

In [ ]:
med.drop('notes', axis=1, inplace=True)

In [ ]:
med['surveyingUser'].value_counts()

In [ ]:
med['surveyingOrganization'].value_counts()

In [ ]:
med.drop(['surveyingUser','surveyingOrganization'],axis=1,inplace=True) # no predictive power

In [ ]:
med.columns

In [ ]:
med['duration'].value_counts()

In [ ]:
med['duration'] = med['duration'].replace('Bdjajbd',None)
med['duration'].value_counts()

In [ ]:
med['seen_doctor'].value_counts()

In [ ]:
med['received_treatment_description'].value_counts()

In [ ]:
med['received_treatment_notes'].value_counts()

In [ ]:
med.drop('received_treatment_notes',axis=1,inplace=True)

In [ ]:
med['part_of_body'].value_counts()

In [ ]:
med['part_of_body'] = med['part_of_body'].replace("",None)

In [ ]:
med['part_of_body'].value_counts()

In [ ]:
med['trauma_induced'].value_counts()

In [ ]:
med['condition_progression'].value_counts()

In [ ]:
med['AssessmentandEvaluation_Surgical'].value_counts()

In [ ]:
med['immediate_follow_up'].value_counts()

In [ ]:
med['part_of_body_description'].value_counts()

In [ ]:
med.drop('part_of_body_description',axis=1,inplace=True)

In [ ]:
med.columns

In [ ]:
med

In [ ]:
medc = med.copy()

In [ ]:
medc = medc.set_index("objectId")
medc

In [ ]:
medc.to_csv('evaluationmedical_noDummies.csv')

In [ ]:
dummycopy = med.select_dtypes(exclude=[np.number]).copy()
obj_id_col = dummycopy['objectId']
dummycopy = dummycopy.drop('objectId',axis=1)

In [ ]:
dummies = pd.get_dummies(dummycopy)
dummies = dummies.reset_index()
dummies

In [ ]:
numeric = med.select_dtypes(include=[np.number])
numeric = numeric.reset_index()
numeric

In [ ]:
master = obj_id_col.to_frame().reset_index().merge(dummies,how='outer', left_index=True, right_index=True)
master

In [ ]:
master.drop(['index_x','index_y'],axis=1,inplace=True)

In [ ]:
master = master.set_index('objectId')
master

In [ ]:
master.to_csv('evaluationmedical.csv')